# Medium claps predictor

## Idea

In [1]:
import os
import pandas
#import torch 
#import torchtext
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import numpy as np
%matplotlib inline

In [142]:
DATA_PATH = './'
# TRAIN_PATH = os.path.join(DATA_PATH, 'articles_train.csv')
TEST_PATH = os.path.join(DATA_PATH, 'articles_test.csv')
SCRAPPING_OUT_PATH = os.path.join(DATA_PATH, 'scrapped_data')

In [2]:
# #За детерминизм!
# SEED = 0xDEAD
# random.seed(SEED)#
# np.random.seed(SEED)
# torch.random.manual_seed(SEED)
# torch.cuda.random.manual_seed_all(SEED)

In [2]:
import pandas as pd

In [4]:
# torch.cuda.is_available()

True

## Data Preparation

In [4]:
train_data = pd.read_csv(TEST_PATH, index_col='id')

In [6]:
train_data.head(3)

,author,reading_time,link,title,text
id,,,,,
3756,Rohit Thakur,8,https://towardsdatascience.com/step-by-step-r-...,Step-by-Step R-CNN Implementation From Scratch...,"Towards Data Science\nOct 18, 2019\nClassifica..."
3757,Giuliano Giacaglia,14,https://towardsdatascience.com/transformers-14...,How Transformers Work. Transformers are a type...,"Towards Data Science\nMar 11, 2019\nIf you lik..."
3758,Darshan Adakane,7,https://towardsdatascience.com/neural-style-tr...,Neural Style Transfer using VGG model | by Dar...,"Towards Data Science\nJan 16, 2020\nIntroducti..."


In [9]:
num_cols = ['reading_time']
cat_cols = ['author', 'link', 'title', 'text']

In [10]:
train_data[num_cols].describe()

,reading_time
count,500.000000
mean,7.880000
std,4.431351
min,1.000000
25%,5.000000
50%,7.000000
75%,10.000000
max,31.000000


In [12]:
# fig, ax = plt.subplots(nrows=1, ncols=len(num_cols), figsize=(20,5))
# fig.suptitle('Ящики с усами для числовых признаков', fontsize=16)
# for i in range(len(num_cols)):
#     ax[i].boxplot(train_data[num_cols[i]])
#     ax[i].set_title(num_cols[i])
# plt.show()

In [13]:
train_data.author.value_counts()

Synced             7
Susan Li           7
Renu Khandelwal    5
Manish Chablani    4
Jonathan Hui       4
                  ..
Sumit Saha         1
Gabe Flomo         1
IPG Media Lab      1
Michael Phi        1
Victor Sanh        1
Name: author, Length: 436, dtype: int64

## Feature Engineering

In [26]:

# ! title clastering (maybe topic modeling LDA)
# ! link parser (some sites are more popular than another)
# ? text parser (only text without titles/dates...)
# ! link scrapper (img per 1000 words, image sizes, count of words)
# ! catboost, lgbm, ridge, random_forest

## Article Scrapper

In [5]:
import requests
import re
import time
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def is_English(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def clean_string(a_string):
    new_string = re.sub(r'‘|’|"|—|“|”|,', '', a_string).strip()
    new_string = re.sub(r'–', ' ', new_string)
    #new_string = re.sub(r'é', 'e', new_string)
    return new_string

def filter_string(a_string):
    if a_string == '.':
        return False
    elif a_string == '':
        return False
    else:
        return True

In [6]:
import re

class ArticleScraper():
    """
    Scrapes all data for an article.
    """
    def __init__(self):
        self.scraper_class = 'ArticleScraper' 

    
    def get_title(self, soup):
        try:
            title = soup.find('h1').text
            return title
        
        except:
            print("Couldn't get title from article.")

    
    def get_subtitle(self, soup):
        try:
            return soup.find('section').find('h2').text
        except:
            print("Couldn't get subtitle from article.")


    def get_publication(self, soup):
        try:
            for div in soup.find_all("div"):
                if div.text == 'Published in':
                    try:
                        return div.next_element.next_element.next_element.find('p').text
                    except:
                        continue
        except:
            print("Couldn't get publication type from article.")


    def get_author(self, soup):
        try:
            return soup.find('h2', class_=r"pw-author-name").text
        except:
            print("Couldn't get title from article.") 


    def get_reading_time(self, soup):
        try:
            return int(soup.find('div', class_=r"pw-reading-time").text.split()[0])
        except:
            print("Couldn't get reading time from article.")


    def get_date(self, soup):
        try:
            date_string = soup.find('p', class_=r"pw-published-date").text 
            return datetime.datetime.strptime(date_string, '%b %d, %Y').strftime('%d/%m/%Y')
        except:
            print("Couldn't get date from article.") 

    def get_followers(self, soup):
        # try:
        #     link = soup.find('div', class_=r"pw-author").find('a').get('href')
        #     if link[0] == '/':
        #         driver.get('https://medium.com' + link)
        #     else:
        #         driver.get(link)
        #     time.sleep(3)
        #     new_soup = BeautifulSoup(driver.page_source, 'lxml')
        # except:
        #     print("Can't go to user's account")
        # try:
        #     link = new_soup.find_all('a', {'role': 'tab'})[-1].get('href')
        #     if link[0] == '/':
        #         driver.get('https://medium.com' + link)
        #     else:
        #         driver.get(link)
        #     time.sleep(3)
        #     new_soup = BeautifulSoup(driver.page_source, 'lxml')
        #     # for button in soup.find_all('button'):
            #     if 'Followers' in button.text or 'followers' in button.text:
            #         fol_string = button.text.split()[0]
            # tens = {'K': 10e2, 'M': 10e5, 'B': 10e8, 'k': 10e2, 'm': 10e5, 'b': 10e8}
            # if (fol_string[-1] != 'K' and fol_string[-1] != 'M' 
            #             and fol_string[-1] != 'k' and fol_string[-1] != 'm'
            #             and fol_string[-1] != 'b' and fol_string[-1] != 'B'):
            #             return int(fol_string)
            # f = lambda x: int(float(x[:-1])*tens[x[-1]])
            # return f(fol_string)
        try:
            for button in soup.find_all('button'):
                if 'Followers' in button.text or 'followers' in button.text:
                    fol_string = button.text.split()[0]
            tens = {'K': 10e2, 'M': 10e5, 'B': 10e8, 'k': 10e2, 'm': 10e5, 'b': 10e8}
            if (fol_string[-1] != 'K' and fol_string[-1] != 'M' 
                        and fol_string[-1] != 'k' and fol_string[-1] != 'm'
                        and fol_string[-1] != 'b' and fol_string[-1] != 'B'):
                        return int(fol_string)
            f = lambda x: int(float(x[:-1])*tens[x[-1]])
            return f(fol_string)
        except:
            print("Can't go to the followers info")    

    
    def get_mean_size(self, soup):
        try:
            pics = soup.find('section').find_all('img')#, width=True, height=True)
            sums = (0,0)
            for pic in pics:
                url = pic.get('src')
                im = Image.open(requests.get(url, stream=True).raw)
                sums = tuple(map(sum, zip(sums, im.size)))
            mean = tuple(ti//len(pics) for ti in sums)
            if len(pics) == 0:
                return (0,0)
            return mean
        except:
            print("Can't get image's sizes")
         

    def count_figures(self, soup):
        try:
            return len(soup.find('section').find_all('img'))
        except:
            print("Can't get amount of pictures")

    def get_pure_text(self, soup):
        try:
            pure_text = ''
            for unparsed in soup.find('section').find_all('p'):
                pure_text += unparsed.text
            return pure_text
        except:
            print("Can't get pure text")
    
    
    def count_words(self, soup):
        try:
           pure_text = self.get_pure_text(soup=soup)
           return len(pure_text.split())
        except:
            print("Can't get words count from the article")

    
    def count_lists(self, soup):
        try:
            return len(soup.find('section').find_all('ol')) + len(soup.find('section').find_all('ul'))
        except:
            print("Can't get lists count")
    
    def bold_text_count(self, soup):
        try:
            return len(soup.find('section').find_all('strong'))
        except:
            print("Can't get bold text count")
    

    def get_blockquotes(self, soup):
        try:
            notes = []
            blockquotes = soup.find_all('blockquote')
            for blockquote in blockquotes:
                notes.append(blockquote.text)
            return notes
        except:
            print("Couldn't get notes from article.")


    def italic_text_count(self, soup):
        try:
            return len(soup.find('section').find_all('em'))
        except:
            print("Can't get italic text count")

    def count_vids(self, soup):
        try:
            yt_vids = []
            article_soup = soup.find('article')
            for figure in article_soup.find_all('figure'):
                yt_soup = figure.find('iframe', src=re.compile('.*youtube.*'))
                if yt_soup == None:
                    continue
                else:
                    yt_vids.append(yt_soup)
                    
            return len(yt_vids)
                    
        except:
            print("Couldn't get YouTube videos.") 
    

    def count_gists(self, soup):
        try:
            gists = []
            article_soup = soup.find('article')
            for fig in article_soup.find_all('figure'):
                gist_soup = fig.find('iframe', title=re.compile('.*\.py'))
                if gist_soup == None:
                    continue
                else:
                    gists.append(gist_soup)
            return len(gists)
        except:
            print("Couldn't get count of gists.") 


    def count_links(self, soup):
        try:
            links = []
            for a in soup.find('section').find_all('a'):
                link = a.get('href')
                if link != None:
                    links.append(link)
            return len(links)
        except:
            print("Couldn't get amount of links.") 

    def count_code_chunks(self, soup):
        try:
            return len(soup.find_all('pre'))
        except:
            print("Couldn't get amount of code chunks from article.")

    def scrape(self, soup):
        im_size = self.get_mean_size(soup)
        if im_size == None:
            im_size = (None, None)
        article_data = {
            "title": self.get_title(soup),
            "publication": self.get_publication(soup),
            #"subtitle": self.get_subtitle(soup),
            "author": self.get_author(soup),
            "followers": self.get_followers(soup),
            "reading_time": self.get_reading_time(soup),
            "n_words": self.count_words(soup),
            "pure_text": self.get_pure_text(soup),
            #"blockquotes": self.get_blockquotes(soup),
            "date": self.get_date(soup),
            "n_code_chunks": self.count_code_chunks(soup),
            "bold_text_count": self.bold_text_count(soup),
            "italic_text_count": self.italic_text_count(soup),
            "mean_image_width": im_size[0],
            "mean_image_height": im_size[1],
            "n_images": self.count_figures(soup),
            "n_lists": self.count_lists(soup),
            #"n_gists": self.count_gists(soup),
            "n_vids": self.count_vids(soup),
            "n_links": self.count_links(soup)
        }
        '''
        # !'bold_text_count',
        # !'count_code_chunks',
        # !'count_figures',
        # !'count_gists',
        # !'count_links',
        # !'count_lists',
        # !'count_vids',
        # !'count_words',
        # !'get_author',
        # !'get_blockquotes',
        # ! 'get_date',
        # !'get_followers',
        # !'get_mean_size',
        # !'get_publication',
        # !'get_pure_text',
        # !'get_reading_time',
        # !'get_subtitle',
        # !'get_title',
        # !'italic_text_count',
        '''
            
        return(article_data)

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import cv2
from tqdm import tqdm, tqdm_notebook
import lxml
from PIL import Image
from langdetect import detect

In [16]:
from selenium.webdriver.edge.options import Options

options = Options()
options.add_argument("--headless")

In [17]:
def scrape_page(link, options):
    page_url = link

    driver = webdriver.Edge(options=options)
    try:
        driver.get(page_url)
    except:
        print('Time out')
        return {
            "title": None,
            "publication": None,
            #"subtitle": self.get_subtitle(soup),
            "author": None,
            "followers": None,
            "reading_time": None,
            "n_words": None,
            "pure_text": None,
            #"blockquotes": self.get_blockquotes(soup),
            "date": None,
            "n_code_chunks": None,
            "bold_text_count": None,
            "italic_text_count": None,
            "mean_image_width": None,
            "mean_image_height": None,
            "n_images": None,
            "n_lists": None,
            #"n_gists": self.count_gists(soup),
            "n_vids": None,
            "n_links": None
        }
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    article_scrapper = ArticleScraper()
    return article_scrapper.scrape(soup=soup)

## Make Scrapped data become better

In [70]:
data = pd.read_csv('test.csv', index_col='idx')

In [71]:
test_data = pd.read_csv(TEST_PATH, index_col='id')

In [72]:
len(test_data) == len(data)

True

In [11]:
data.sample(3)

,title,publication,author,followers,reading_time,n_words,pure_text,date,n_code_chunks,bold_text_count,italic_text_count,mean_image_width,mean_image_height,n_images,n_lists,n_vids,n_links
idx,,,,,,,,,,,,,,,,,
4045,Object Recognition with OpenCV on Android,NaN,Akshika Wijesundara,465.0,6.0,763.0,This article is for a person who has some know...,20/12/2016,4,236.0,49.0,1400.0,786.0,1.0,1.0,0.0,9.0
4178,Understanding Attention Mechanism,NaN,Shashank Yadav,85.0,5.0,800.0,Attention mechanism for sequence modelling was...,06/02/2019,0,28.0,8.0,1400.0,625.0,10.0,1.0,0.0,1.0
4052,Sentiment analysis using RNNs(LSTM),Towards Data Science,Manish Chablani,1700.0,4.0,445.0,Here we use the example of reviews to predict ...,21/06/2017,2,0.0,0.0,489.0,578.0,1.0,0.0,0.0,4.0


In [73]:
is_null = data.isnull()
row_with_null = is_null.all(axis=1)
rows_with_null = data[row_with_null].index
with tqdm(rows_with_null, unit="page") as tepoch:
    for idx in tepoch:
        new_row = scrape_page(test_data.loc[idx].link, options)
        data.loc[idx,
                data.columns] = new_row

0page [00:00, ?page/s]


In [74]:
def scrape_column(column):
    print('Before repeated scrapping: ', data[column].isna().sum())
    indeces = data[data[column].isna() == True].index
    with tqdm(indeces, unit="page") as tepoch:
        for idx in tepoch:
            new_row = scrape_page(test_data.loc[idx].link, options)
    #     print(new_row['pure_text'])
            data.loc[idx,
                column] = new_row[column]
    print('After scrapping: ', data[column].isna().sum())

In [75]:
scrape_column('title')

Before repeated scrapping:  7


  0%|          | 0/7 [00:00<?, ?page/s]

Can't get image's sizes
Couldn't get title from article.


 14%|█▍        | 1/7 [00:09<00:56,  9.41s/page]

Couldn't get title from article.


...

Couldn't get title from article.


 71%|███████▏  | 5/7 [00:50<00:20, 10.49s/page]

Can't get image's sizes
Couldn't get title from article.


100%|██████████| 7/7 [01:16<00:00, 10.96s/page]

After scrapping:  6


In [76]:
indeces_title = data[data['title'].isna() == True].index

In [77]:
for idx in indeces_title:
    data.iat[idx-3756, data.columns.get_loc('title')]  = test_data.loc[idx].title.split('|')[0]

In [78]:
data['title'].isna().sum()

0

In [79]:
scrape_column('publication')

Before repeated scrapping:  116


  0%|          | 0/116 [00:00<?, ?page/s]

Can't get image's sizes


  4%|▍         | 5/116 [01:02<24:38, 13.32s/page]

Can't get image's sizes
Couldn't get title from article.


...

Can't get image's sizes


 97%|█████████▋| 112/116 [24:41<00:51, 12.75s/page]

Can't get image's sizes


100%|██████████| 116/116 [25:54<00:00, 13.40s/page]

After scrapping:  115


In [80]:
data.publication = data.publication.fillna('Medium').values

In [81]:
data['publication'].isna().sum()

0

In [82]:
scrape_column('author')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.24s/page]

After scrapping:  0


In [83]:
scrape_column('followers')

Before repeated scrapping:  4


  0%|          | 0/4 [00:00<?, ?page/s]

Can't go to the followers info


 25%|██▌       | 1/4 [00:14<00:44, 14.89s/page]

Can't go to the followers info


 50%|█████     | 2/4 [00:28<00:27, 13.96s/page]

Can't get image's sizes
Can't go to the followers info


100%|██████████| 4/4 [00:53<00:00, 13.37s/page]

After scrapping:  3


In [84]:
indeces_followers = data[data['followers'].isna() == True].index

In [85]:
for idx in indeces_followers:
    data.loc[idx,
                'followers'] = 1

In [86]:
data.followers.isna().sum()

0

In [87]:
scrape_column('reading_time')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.82s/page]

After scrapping:  0


In [88]:
scrape_column('n_words')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.75s/page]

After scrapping:  0


In [89]:
scrape_column('pure_text')

Before repeated scrapping:  2


100%|██████████| 2/2 [00:47<00:00, 23.96s/page]

After scrapping:  0


In [90]:
scrape_column('date')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.83s/page]

After scrapping:  0


In [91]:
scrape_column('n_code_chunks')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [92]:
scrape_column('bold_text_count')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.35s/page]

After scrapping:  0


In [93]:
scrape_column('italic_text_count')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.28s/page]

After scrapping:  0


In [94]:
scrape_column('n_images')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:17<00:00, 17.61s/page]

After scrapping:  0


In [112]:
data.loc[data[data['n_images'] == 0].index, ['mean_image_width', 'mean_image_height']] = [0, 0]

In [116]:
data.mean_image_width.isna().sum(), data.mean_image_height.isna().sum()

(3, 3)

In [117]:
indeces_width = data[data['mean_image_width'].isna() == True].index

In [125]:
data.loc[indeces_width, 'mean_image_width'] = int(np.mean(data.mean_image_width))

In [123]:
indeces_height = data[data['mean_image_height'].isna() == True].index

In [127]:
data.loc[indeces_height, 'mean_image_height'] = int(np.mean(data.mean_image_height))

In [129]:
data.mean_image_width.isna().sum(), data.mean_image_height.isna().sum()

(0, 0)

In [131]:
scrape_column('n_images')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [132]:
scrape_column('n_lists')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.96s/page]

After scrapping:  0


In [137]:
scrape_column('n_vids')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:16<00:00, 16.53s/page]

After scrapping:  0


In [138]:
scrape_column('n_links')

Before repeated scrapping:  1


100%|██████████| 1/1 [00:18<00:00, 18.87s/page]

After scrapping:  0


In [139]:
languages = []

for text in data.pure_text:
    try:
        languages.append(detect(text[:50]))
    except:
        print('Sorry')
        languages.append(None)

In [140]:
data['language'] = languages

In [141]:
data.sample(2)

,title,publication,author,followers,reading_time,n_words,pure_text,date,n_code_chunks,bold_text_count,italic_text_count,mean_image_width,mean_image_height,n_images,n_lists,n_vids,n_links,language
idx,,,,,,,,,,,,,,,,,,
3780,The best explanation of Convolutional Neural N...,TechnologyMadeEasy,Harsh Pokharna,2600.0,5.0,726.0,CNNs have wide applications in image and video...,28/07/2016,0,14.0,14.0,708.0,376.0,10.0,0.0,0.0,3.0,en
3986,Everything You Need to Know About Artificial N...,"Technology, Invention, App, and More",Josh,42000.0,9.0,1986.0,The year 2015 was a monumental year in the fie...,28/12/2015,0,26.0,6.0,578.0,375.0,9.0,0.0,0.0,32.0,en


In [143]:
data.to_csv(os.path.join(SCRAPPING_OUT_PATH, 'scrapped_test.csv'))
# data.to_csv(os.path.join(SCRAPPING_OUT_PATH, 'scrapped_train.csv'))